<a href="https://colab.research.google.com/github/pandurangpatil/erav4-backpropbay/blob/main/session7-ass/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training CIFAR-100 Model (Google Colab)
This notebook trains WideResNet-28-10 on CIFAR-100 with batch_size=256 and epochs=100

## Clone Repository

In [4]:
# Clone the repository
!git clone https://github.com/pandurangpatil/erav4-backpropbay.git

Cloning into 'erav4-backpropbay'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 90 (delta 39), reused 41 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (90/90), 1.38 MiB | 9.16 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [ ]:
# Navigate to the session8-more-trials directory
%cd erav4-backpropbay/session8-more-trials

In [6]:
# Install dependencies from requirements.txt
!pip install -r requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Install Required Dependencies

In [ ]:
!pip install torchsummary tqdm albumentations huggingface_hub plotille -q

## Setup HuggingFace Token (Optional)

In [ ]:
# Setup HuggingFace token from Colab secrets (optional)
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    HF_REPO_ID = 'your-username/cifar100-wideresnet-session8'  # Change this to your repo
    print("✓ HuggingFace token retrieved from Colab secrets")
    print(f"Will upload to: {HF_REPO_ID}")
except Exception as e:
    print(f"Warning: Could not retrieve HF_TOKEN from secrets: {e}")
    print("Training will proceed without HuggingFace upload")
    HF_TOKEN = None
    HF_REPO_ID = None

## Check GPU and Import Libraries

## Initialize and Train CIFAR-100 Model

In [ ]:
# Create trainer with CIFAR-100 configuration
trainer = CIFARTrainer(
    model_module_name='model',
    epochs=100,
    batch_size=256,
    use_mixup=True,
    mixup_alpha=0.2,
    label_smoothing=0.1,
    use_amp=True,
    gradient_clip=1.0,
    warmup_epochs=5,
    checkpoint_epochs=[10, 20, 25, 30, 40, 50, 60, 75, 90],
    hf_token=HF_TOKEN,
    hf_repo_id=HF_REPO_ID
)

print("\n" + "="*70)
print("Starting CIFAR-100 WideResNet-28-10 Training")
print("="*70)
print("Configuration:")
print("  - Model: WideResNet-28-10 (36.5M parameters)")
print("  - Dataset: CIFAR-100 (100 classes)")
print("  - Batch Size: 256")
print("  - Epochs: 100")
print("  - Optimizer: SGD (lr=0.01, momentum=0.9, weight_decay=1e-3)")
print("  - Scheduler: CosineAnnealingWarmRestarts (T_0=25)")
print("  - Warmup: 5 epochs (0.01 → 0.1)")
print("  - MixUp: Enabled (alpha=0.2)")
print("  - Label Smoothing: 0.1")
print("  - Mixed Precision: Enabled")
print("  - Gradient Clipping: 1.0")
print("  - Early Stopping: patience=15")
if HF_REPO_ID:
    print(f"  - HuggingFace Upload: Enabled ({HF_REPO_ID})")
else:
    print("  - HuggingFace Upload: Disabled")
print("="*70)

In [ ]:
# Run training
best_accuracy = trainer.run()

## Training Complete - Summary

In [ ]:
print("\n" + "="*70)
print("TRAINING COMPLETED - CIFAR-100 WideResNet-28-10")
print("="*70)
print(f"Best Test Accuracy: {best_accuracy:.2f}%")
print(f"Best Epoch: {trainer.best_epoch}")
print(f"Total Epochs Trained: {len(trainer.train_accuracies)}")
print(f"\nFinal Metrics:")
print(f"  - Train Accuracy: {trainer.train_accuracies[-1]:.2f}%")
print(f"  - Test Accuracy: {trainer.test_accuracies[-1]:.2f}%")
print(f"  - Train Loss: {trainer.train_losses[-1]:.4f}")
print(f"  - Test Loss: {trainer.test_losses[-1]:.4f}")
print(f"\nCheckpoints saved in: ./checkpoints/")
if HF_REPO_ID:
    print(f"HuggingFace Model: https://huggingface.co/{HF_REPO_ID}")
print("="*70)

In [11]:
print("\n" + "="*60)
print("TRAINING COMPLETED")
print("="*60)
print(f"Model: model")
print(f"Final Test Accuracy: {final_accuracy:.2f}%")
print("="*60)


TRAINING COMPLETED
Model: model
Final Test Accuracy: 87.28%
